In [1]:
import pandas as pd
import gc
import numpy as np
import lightgbm as lgb
import pickle as pickle
from datetime import datetime, timedelta
from scripts.feature.get_feature import get_short_lag_feature


/home/yuto/m5-forecasting-accuracy/.venv/lib/python3.7/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
2020-04-14 22:10:41,003 INFO luigi-interface : Loaded ['luigi.cfg']


In [2]:
gc.enable()

In [3]:
pd.options.display.max_columns = 50

In [4]:
debug = False

if not debug:
    df = pd.read_pickle("./resources/feature/feature.pkl")
else:
    df = pd.read_csv("df.csv")
    df = df.rename(columns={"sales": "demand"})
    df = df.set_index(["date", "id"])
df.head()

,,demand,lag_7,lag_28,rmean_7_7,rmean_28_7,rmean_7_28,rmean_28_28,snap_CA,snap_TX,snap_WI,year,month,week,day,dayofweek,item_id,dept_id,cat_id,store_id,state_id,event_name_1,event_type_1,event_name_2,event_type_2,sell_price
id,date,,,,,,,,,,,,,,,,,,,,,,,,,
HOBBIES_1_001_CA_1_validation,2013-06-02,0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,1,0,1,2013,6,22,2,6,1437,3,1,0,0,29,4,4,2,-999.00
HOBBIES_1_002_CA_1_validation,2013-06-02,0,0.0,1.0,0.142857,0.285714,0.285714,0.178571,1,0,1,2013,6,22,2,6,1438,3,1,0,0,29,4,4,2,3.97
HOBBIES_1_003_CA_1_validation,2013-06-02,0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,1,0,1,2013,6,22,2,6,1439,3,1,0,0,29,4,4,2,-999.00
HOBBIES_1_004_CA_1_validation,2013-06-02,4,3.0,1.0,1.714286,1.428571,1.571429,1.642857,1,0,1,2013,6,22,2,6,1440,3,1,0,0,29,4,4,2,4.64
HOBBIES_1_005_CA_1_validation,2013-06-02,0,1.0,0.0,1.571429,0.285714,0.964286,0.857143,1,0,1,2013,6,22,2,6,1441,3,1,0,0,29,4,4,2,3.08


In [5]:
cat_feats = ["item_id", "dept_id", "store_id", "cat_id", "state_id"] + [
    "event_name_1",
    "event_name_2",
    "event_type_1",
    "event_type_2",
]
if debug:
    useless_cols = ["demand", "d", "wm_yr_wk", "weekday"]
    train_cols = df.columns[~df.columns.isin(useless_cols)]
else:
    train_cols = df.drop(columns="demand").columns

In [6]:
X_train = df[train_cols]
y_train = df["demand"]

In [7]:
%%time

np.random.seed(777)

fake_valid_inds = np.random.choice(
    X_train.reset_index().index.values, 2_000_000, replace=False
)
train_inds = np.setdiff1d(X_train.reset_index().index.values, fake_valid_inds)
print("train_inds")
train_data = lgb.Dataset(
    X_train.iloc[train_inds],
    label=y_train.iloc[train_inds],
    categorical_feature=cat_feats,
    free_raw_data=False,
)
print("train_data end")
fake_valid_data = lgb.Dataset(
    X_train.iloc[fake_valid_inds],
    label=y_train.iloc[fake_valid_inds],
    categorical_feature=cat_feats,
    free_raw_data=False,
)

train_inds
train_data end
CPU times: user 10.7 s, sys: 4.61 s, total: 15.3 s
Wall time: 15.3 s


In [8]:
test = df.reset_index().query("'2016-04-25' <= date").copy()
test['date'] = pd.to_datetime(test['date'])
test.head()

,id,date,demand,lag_7,lag_28,rmean_7_7,rmean_28_7,rmean_7_28,rmean_28_28,snap_CA,snap_TX,snap_WI,year,month,week,day,dayofweek,item_id,dept_id,cat_id,store_id,state_id,event_name_1,event_type_1,event_name_2,event_type_2,sell_price
32166950,HOBBIES_1_001_CA_1_validation,2016-04-25,0,1.0,1.0,0.857143,1.142857,0.964286,1.035714,0,0,0,2016,4,17,25,0,1437,3,1,0,0,29,4,4,2,8.38
32166951,HOBBIES_1_002_CA_1_validation,2016-04-25,0,0.0,1.0,0.000000,0.714286,0.178571,0.357143,0,0,0,2016,4,17,25,0,1438,3,1,0,0,29,4,4,2,3.97
32166952,HOBBIES_1_003_CA_1_validation,2016-04-25,0,1.0,0.0,1.285714,0.285714,0.464286,0.535714,0,0,0,2016,4,17,25,0,1439,3,1,0,0,29,4,4,2,2.97
32166953,HOBBIES_1_004_CA_1_validation,2016-04-25,0,4.0,0.0,1.857143,3.000000,2.071429,1.892857,0,0,0,2016,4,17,25,0,1440,3,1,0,0,29,4,4,2,4.64
32166954,HOBBIES_1_005_CA_1_validation,2016-04-25,0,0.0,1.0,0.857143,0.714286,1.071429,1.107143,0,0,0,2016,4,17,25,0,1441,3,1,0,0,29,4,4,2,2.88


In [9]:
del df, X_train, y_train, fake_valid_inds, train_inds
gc.collect()

29

In [10]:
params = {
    "objective": "poisson",
    "metric": "rmse",
    "force_row_wise": True,
    "learning_rate": 0.075,
    "sub_row": 0.75,
    "bagging_freq": 1,
    "lambda_l2": 0.1,
    "metric": ["rmse"],
    "verbosity": 1,
    "num_iterations": 1200,
    "num_leaves": 128,
    "min_data_in_leaf": 100,
}

In [11]:
load = True
model_name = "my_lgb.pkl"
if load:
    with open(model_name, "rb") as f:
        m_lgb = pickle.load(f)
else:
    m_lgb = lgb.train(params, train_data, valid_sets=[fake_valid_data], verbose_eval=100)
    with open(model_name, "wb") as f:
        pickle.dump(m_lgb, f)

In [ ]:
%%time

# alphas = [1.028, 1.023, 1.018]
alphas = [1.028]
weights = [1 / len(alphas)] * len(alphas)
sub = 0.0
fday = datetime(2016, 4, 25)
max_lags = 57

for icount, (alpha, weight) in enumerate(zip(alphas, weights)):

    te = test.copy()
    print("copied, create feature")
    te = get_short_lag_feature(te)
    cols = [f"F{i}" for i in range(1, 29)]

    for tdelta in range(0, 28):
        day = fday + timedelta(days=tdelta)
        print(tdelta, day)
        tst = te[(te.date >= day - timedelta(days=max_lags)) & (te.date <= day)].copy()
        #         create_step_by_step_feature(tst, train_cols)
        tst = get_short_lag_feature(tst)
        tst = tst.loc[tst.date == day, train_cols]
        te.loc[te.date == day, "sales"] = alpha * m_lgb.predict(
            tst
        )  # magic multiplier by kyakovlev

    te_sub = te.loc[te.date >= fday, ["id", "sales"]].copy()

    cols = [f"F{i}" for i in range(1, 29)]
    te_sub["F"] = [f"F{rank}" for rank in te_sub.groupby("id")["id"].cumcount() + 1]
    te_sub = te_sub.set_index(["id", "F"]).unstack()["sales"][cols].reset_index()
    te_sub.fillna(0.0, inplace=True)
    te_sub.sort_values("id", inplace=True)
    te_sub.reset_index(drop=True, inplace=True)
    te_sub.to_csv(f"submission_{icount}.csv", index=False)
    if icount == 0:
        sub = te_sub
        sub[cols] *= weight
    else:
        sub[cols] += te_sub[cols] * weight
    print(icount, alpha, weight)


sub2 = sub.copy()
sub2["id"] = sub2["id"].str.replace("validation$", "evaluation")
sub = pd.concat([sub, sub2], axis=0, sort=False)
sub.to_csv("submission.csv", index=False)

copied, create feature
0 2016-04-25 00:00:00
1 2016-04-26 00:00:00
2 2016-04-27 00:00:00
3 2016-04-28 00:00:00
4 2016-04-29 00:00:00
5 2016-04-30 00:00:00
6 2016-05-01 00:00:00
7 2016-05-02 00:00:00
8 2016-05-03 00:00:00
9 2016-05-04 00:00:00
10 2016-05-05 00:00:00
11 2016-05-06 00:00:00
12 2016-05-07 00:00:00


In [ ]:
print("hello")